# Imports

In [153]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
import numpy as np
import copy
import time
import snakeviz
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist
import random
from sklearn.datasets import load_diabetes
assert pyro.__version__.startswith('1.8.4')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

In [154]:
from sklearn import linear_model

# Load Datasets

### Random Dataset

In [174]:
def generate_stdp_dataset(dim, num_examples, min_value, max_value):
    X = np.random.random((num_examples + 1, dim)) * (max_value - min_value) + min_value
    beta = np.random.random((dim)) * (max_value - min_value) + min_value

    noise = np.random.normal(0, np.sqrt(max_value - min_value), num_examples + 1)
    Y = X[:num_examples + 1] @ beta + noise

    X = np.asfortranarray(X)
    Y = np.asfortranarray(Y)
    X /= np.linalg.norm(X, axis=0)
    Y = (Y - Y.mean()) / Y.std()
    Y = Y * max_value

    Y = Y/np.linalg.norm(Y)

    return X, Y, beta

In [182]:
X, Y, beta = generate_stdp_dataset(100, 100, 0, 1)

### Diabetes Dataset

In [171]:
X, Y = load_diabetes(return_X_y = True)

In [178]:
print(X[0:3])
print(np.linalg.norm(X))
print(Y[0:3])
print(Y.mean())
print(Y.std())

[[0.18425529 0.36282919 0.12248007]
 [0.18438486 0.41843389 0.23002136]
 [0.05056745 0.04801674 0.41256061]]
1.7320508075688772
[ 0.66760437 -0.63626265 -0.05940061]
2.0185873175002847e-17
0.30151134457776363


### Normalize X and Y

In [183]:
Y = (Y - Y.mean()) / Y.std()
print(Y[0:3])
X = X / np.linalg.norm(X)
print(X[0:3])

[-0.08854301  0.18771884 -1.15194511]
[[1.23369942e-02 1.10714128e-02 3.36875906e-03 1.52335770e-02
  8.81843816e-03 1.13205919e-02 2.80707645e-03 1.53106857e-03
  2.29692850e-03 8.01404924e-03 1.41844578e-02 6.46349722e-03
  1.48955589e-02 1.52507784e-02 9.27055213e-03 2.86867258e-03
  9.48210382e-03 2.40919016e-03 3.31625332e-03 9.11558112e-03
  2.84497944e-03 9.68484171e-03 1.24691227e-02 5.74841346e-03
  1.59205167e-02 1.09303740e-02 1.14351426e-02 1.56571474e-02
  1.14607191e-02 1.24244405e-02 1.52878966e-02 1.31947387e-02
  1.12956883e-02 1.38244396e-02 1.33482648e-02 1.49576619e-02
  7.81153713e-03 1.21660859e-02 2.82761044e-04 9.60874765e-03
  2.20273717e-03 1.10216115e-02 5.42463315e-03 4.59682123e-03
  9.02209787e-03 1.07307544e-02 6.11339532e-03 1.62219780e-02
  8.06222692e-03 8.32210973e-03 1.25382640e-03 2.80676445e-03
  1.03621548e-02 2.83218160e-03 7.24867213e-03 1.19819763e-02
  1.64003167e-02 4.24912510e-03 7.95779637e-03 1.38827701e-02
  1.29542764e-02 1.54626418e-02 

In [184]:
print(X)
print(Y)

[[1.23369942e-02 1.10714128e-02 3.36875906e-03 ... 1.14726187e-02
  5.08932800e-03 1.01253437e-03]
 [8.87809186e-03 1.26559807e-02 1.44054601e-02 ... 1.98727583e-03
  1.39376827e-02 7.61330947e-03]
 [7.99025123e-03 1.30332911e-02 1.02413174e-02 ... 6.24423093e-03
  8.08749284e-04 1.09718581e-04]
 ...
 [5.66635755e-04 4.91321107e-03 8.59975387e-03 ... 2.37606961e-03
  1.34129820e-03 8.58239756e-03]
 [4.57075163e-05 3.36557181e-03 3.52825280e-03 ... 5.99018170e-03
  6.66847153e-04 1.17820209e-03]
 [1.28771312e-02 1.17788107e-02 1.32797460e-03 ... 2.05025855e-03
  8.08628473e-03 4.05003898e-03]]
[-8.85430117e-02  1.87718844e-01 -1.15194511e+00  6.78023035e-01
  4.71750906e-01 -2.01865142e-01  9.89222305e-01  1.21649895e+00
  1.68771913e-03 -4.07835279e-01 -1.05579453e-01  1.68393654e+00
 -2.07028853e+00 -1.13561473e+00 -1.01761886e+00  5.33274109e-02
 -8.41758479e-02  1.86604264e-02  3.86638565e+00 -1.17517833e-02
 -1.84554510e+00  9.36253996e-01 -1.80171524e-01 -5.73655276e-01
 -3.590448

In [185]:
print(len(X[0]))

100


### Set up X, Y train

In [186]:
X_train = copy.deepcopy(X)
Y_train = copy.deepcopy(Y[:len(Y) - 1])
# X_train = [torch.tensor(member) for member in X_train]
# Y_train = [torch.tensor(member) for member in Y_train]
dim = len(X[0])

In [187]:
np.std(Y_train)

1.004824689354094

# Model and Guide Setup

In [188]:
global prev_mu_q

prev_mu_q = torch.zeros(dim, dtype=torch.float64)

In [189]:
std0 = torch.eye(dim, dtype=torch.float64) * 0.3
def model(data):
    # define the hyperparameters that control the Beta prior
    mu0 = torch.zeros(dim, dtype=torch.float64)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.MultivariateNormal(mu0, std0))
    # loop over the observed data
    subset = random.sample(data, int(len(data) / dim))
    for i in range(len(subset)):
        pyro.sample("obs_{}".format(i), dist.Normal(f.dot(data[i][0]), 0.3), obs=data[i][1])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    mu_q = pyro.param("mu_q", copy.deepcopy(prev_mu_q))
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.MultivariateNormal(mu_q, std0))

In [190]:
def train_SVI(D_hat, n_steps):
    # setup the optimizer
    adam_params = {"lr": 0.005, "betas": (0.90, 0.999)}
    optimizer = Adam(adam_params)

    # setup the inference algorithm
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    # do gradient steps
    for step in range(n_steps):
        loss = svi.step(D_hat)
        if loss < 1e-5:
            break
    
    breakpoint()
    mu_q = pyro.param("mu_q")
    return mu_q

# Run Sklearn on Data

In [191]:
y_hat = max(Y_train)
y_bottom = min(Y_train)
print(y_hat)
print(y_bottom)
conformal_set = []
decrease_size = 0.1
start = time.time()
while y_hat >= y_bottom:
    pyro.clear_param_store()
    # Create D_hat
    # SVI shape: [(tensor(X), tensor(Y)), (tensor(X), tensor(Y)) ...]
    # Sklearn: [[array X1, array X2, array X3 .. ], [array Y1, array Y2, array Y3 ...]]
    D_hat[0] = X_train
    D_hat[1] = np.append(Y_train, y_hat)
    
    # Train sklearn model
    clf = linear_model.Lasso(alpha=0.1, tol=1e-8)
    clf.fit(D_hat[0], D_hat[1])
    mu_q = clf.coef_
    prev_mu_q = mu_q
    
    # Calculate rank of y_hat
    rank = [(abs(sum(D_hat[0][i] * mu_q) - D_hat[1][i])) for i in range(len(D_hat[0]))]
    y_hat_rank = rank[-1]
    
    # Add to conformal set if in not in bottom 10 percent of probabilities
    if np.count_nonzero(y_hat_rank > rank) / len(rank) < 0.9:
        conformal_set.append(copy.deepcopy(y_hat))
        print(f"{y_hat} Added")
    else:
        print(f"{y_hat} Not added")
        
    y_hat -= decrease_size
conformal_set = [min(conformal_set), max(conformal_set)]
end = time.time()

3.866385654267805
-2.070288532508463
3.866385654267805 Not added
3.766385654267805 Not added
3.666385654267805 Not added
3.5663856542678047 Not added
3.4663856542678047 Not added
3.3663856542678046 Not added
3.2663856542678045 Not added
3.1663856542678044 Not added
3.0663856542678043 Not added
2.966385654267804 Not added
2.866385654267804 Not added
2.766385654267804 Not added
2.666385654267804 Not added
2.566385654267804 Not added
2.4663856542678038 Not added
2.3663856542678037 Not added
2.2663856542678036 Not added
2.1663856542678035 Not added
2.0663856542678034 Not added
1.9663856542678033 Not added
1.8663856542678032 Not added
1.7663856542678031 Not added
1.666385654267803 Not added
1.566385654267803 Added
1.4663856542678029 Added
1.3663856542678028 Added
1.2663856542678027 Added
1.1663856542678026 Added
1.0663856542678025 Added
0.9663856542678025 Added
0.8663856542678026 Added
0.7663856542678026 Added
0.6663856542678026 Added
0.5663856542678026 Added
0.46638565426780265 Added
0.366

In [192]:
print(f"Conformal Set: [{float(conformal_set[0])}, {float(conformal_set[1])}]")
print(f"Length: {float(conformal_set[1] - conformal_set[0])}")
print(f"Y[-1]: {Y[-1]}")
if Y[-1] >= conformal_set[0] and Y[-1] <= conformal_set[1]:
    print(f"Y[-1] is covered")
else:
    print("Y[-1] is Not covered")
print(f"Elapsed Time: {end - start}")

Conformal Set: [-1.6336143457321979, 1.566385654267803]
Length: 3.200000000000001
Y[-1]: 0.18002850831690767
Y[-1] is covered
Elapsed Time: 0.09563803672790527
